In [1]:
from pylab import *
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import sisl
import ipyvolume as ipv
#Define Variables:
#####################################################################################################
#Define electrode Label:
label = "au_hcp" #ABA array
#Define lattice vectors:
a0 = 4.08
a1 = a0*np.array([1.0,0.0,0.0])
a2 = a0*np.array([0.0,1.0,0.0])
a3 = a0*np.array([0.0,0.0,1.0])
#Basis vectors (fractional if you want):
b1 = 0.0*a1 + 0.0*a2 + 0.0*a3
b2 = 0.0*a1 + 0.5*a2 + 0.5*a3
b3 = 0.5*a1 + 0.0*a2 + 0.5*a3
b4 = 0.5*a1 + 0.5*a2 + 0.0*a3
ntraslationsx, ntraslationsy, ntraslationsz  = int(1), int(2), int(2)
#########################################################################################################
molecule_geometry_file = "viologen_aniline_0.006.xyz"
right_electrode = int(input("Would you like to add the right electrode?:\n"+"(1) Yes\n" + "(2) No\n"))
traslation_rotation = int(input('What would you like to do?:\n'+'(1) Scan distance\n'+'(2) Scan rotation\n'))
if traslation_rotation == 1:
    scan_d_min = float(input('Minium distance for scanning:\t'))
    scan_d_max = float(input('Maxium distance for scanning:\t'))
    n_points_scan = int(input('Number of points for scanning:\t'))
    x_scan = np.linspace(scan_d_min,scan_d_max,n_points_scan)
elif traslation_rotation == 2:
    scan_r_min = float(input('Minium angle for scanning:\t'))
    scan_r_max = float(input('Maxium angle for scanning:\t'))
    n_points_scan = int(input('Number of points for scanning:\t'))
    angle_rotationyz = np.linspace(scan_r_min,scan_r_max,n_points_scan)
    x_fix1 = float(input("Indicate equilibrium distance with left electrode:\t"))
    if right_electrode == 1:
        x_fix2 = float(input("Indicate equilibrium distance with right electrode:\t"))
    else:
        pass

Would you like to add the right electrode?:
(1) Yes
(2) No
1
What would you like to do?:
(1) Scan distance
(2) Scan rotation
1
Minium distance for scanning:	1.7
Maxium distance for scanning:	5
Number of points for scanning:	10


In [2]:
################ Generating Supercell #############################################################
basis = [b1,b2,b3,b4]
np_lattice = (ntraslationsx+1)*(ntraslationsy+1)*(ntraslationsz+1) # Número de puntos en la supercelda
pointsR = np.zeros([np_lattice,3],dtype=np.float64)
count1 = 0
for n1 in range(0,int(ntraslationsx+1)):
  for n2 in range(0,int(ntraslationsy+1)):
    for n3 in range(0,int(ntraslationsz+1)):
      pointsR[count1,:] = n1*a1 + n2*a2 + n3*a3
      count1 = count1 + 1 
Natoms = np_lattice*len(basis)   # Número de átomos en la supercelda
supercell_xyz = np.zeros([Natoms,3],dtype=np.float64)
count2 = 0
for k in range(len(pointsR)):
  for l in range(len(basis)):
    supercell_xyz[count2,:]=pointsR[k] + basis[l]
    count2 = count2 + 1
VolR = np.dot(a1,np.cross(a2,a3))
print("System:" + "\t" + str(label)+str(Natoms)+ "_electrode" + "\n"+ "Number of atoms:" + "\t" + str(Natoms) + "\t"+ "Vol. Cell:" + 1*"\t" +str("{:.2f}".format(VolR)))
xmax_sc, ymax_sc, zmax_sc = np.amax(supercell_xyz[:,0]), np.amax(supercell_xyz[:,1]), np.amax(supercell_xyz[:,2])
print("Max Supercell Distance in x:\t","{:.5f}".format(xmax_sc))
print("Max Supercell Distance in y:\t","{:.5f}".format(ymax_sc)) 
print("Max Supercell Distance in z:\t","{:.5f}".format(zmax_sc))
#######################################################################################################################
a1_elec = [xmax_sc + 0.5*a0, 0.0, 0.0]
a2_elec = [0.0,3*ymax_sc,0.0]
a3_elec = [0.0,0.0,3*zmax_sc]
lattice_vectors_electrode = "cell= " + str("{:.5f}".format(a1_elec[0])) + " 0.00000 0.00000 " + "0.00000 " + str("{:.5f}".format(a2_elec[1])) + " 0.00000" + " 0.00000 0.00000 " + str("{:.5f}".format(a3_elec[2])) + " nsc= 1 1 1" + "\n"
print("\nUse the next lattice vectors for electrode:\n" + lattice_vectors_electrode)
electrode = open(str(label)+str(Natoms)+"_electrode"+".xyz","w")
electrode.write(str(Natoms) + "\n")
electrode.write(lattice_vectors_electrode)
for atom in range(Natoms):
    electrode.write("Au" + "  " + str("{:.5f}".format(supercell_xyz[atom,0])) + "  " +str("{:.5f}".format(supercell_xyz[atom,1])) + "  " +str("{:.5f}".format(supercell_xyz[atom,2])) + "\n")
electrode.close()
#######################################################################################################################
FileElec = str(label)+str(Natoms)+"_electrode"+".xyz"
Elec_leftXYZ = sisl.io.xyzSile(FileElec,mode='r')
Elec_elec    = Elec_leftXYZ.read_geometry()
Elec_elec.write("STRUCT_ELEC.fdf")
Elec_elec.write("STRUCT_ELEC.xyz")

System:	au_hcp72_electrode
Number of atoms:	72	Vol. Cell:	67.92
Max Supercell Distance in x:	 6.12000
Max Supercell Distance in y:	 10.20000
Max Supercell Distance in z:	 10.20000

Use the next lattice vectors for electrode:
cell= 8.16000 0.00000 0.00000 0.00000 30.60000 0.00000 0.00000 0.00000 30.60000 nsc= 1 1 1



In [3]:
def Readxyz(File):
    x,y,z    = loadtxt(File,usecols=(1,2,3),unpack=True,skiprows=2)
    atom     = loadtxt(File,usecols=(0,),unpack=True,skiprows=2,dtype="str")
    natoms   = len(x)
    Data     = {"label":File,
                "x":x,
                "y":y,
                "z":z,
                "atom":atom,
                "natoms":natoms}
    #print ("System: ", File)
    #print ("Número de átomos: {0}".format(natoms))
    #print ("Tipos de atomos:",unique(atom))
    return Data
###################################################################################################
molecule = Readxyz(molecule_geometry_file)
molecule_label = molecule["label"].replace("_0.006.xyz","")
Natoms2 = molecule["natoms"]
x_coord, y_coord, z_coord = molecule["x"], molecule["y"], molecule["z"]
atom_s = molecule["atom"]    
###################### Defining molecule midpoint as axis rotation ################################
xmax_mol, xmin_mol = np.amax(x_coord), np.amin(x_coord)
ymax_mol, ymin_mol = np.amax(y_coord), np.amin(y_coord)
zmax_mol, zmin_mol = np.amax(z_coord), np.amin(z_coord)
#########################################################################################
def rotation_2D(angle,x1,x2,xref1,xref2):
    angle = angle*math.pi/180
    x1n = (x1-xref1)*math.cos(angle) - (x2-xref2)*math.sin(angle)
    x2n = (x1-xref1)*math.sin(angle) + (x2-xref2)*math.cos(angle)
    return x1n, x2n
#########################################################################################
angle_rotationxy = 180 # zero for no rotation
if angle_rotationxy == 180:
    xmax_mol, xmin_mol = xmin_mol,xmax_mol
elif angle_rotationxy == 90:
    xmax_mol, xmin_mol = ymin_mol, ymax_mol
else:
    pass
##Defining molecule midpoint:
xm, ym, zm =  0.5*(xmax_mol+xmin_mol), 0.5*(ymax_mol+ymin_mol), 0.5*(zmax_mol+zmin_mol)
###################################################################
x_rxy, y_rxy = rotation_2D(angle_rotationxy,x_coord,y_coord,xm,ym)
###################################################################
electrode_atom_ref = int(input("Electrode reference atom:\t"))
molecule_atom_ref = int(input("Molecule reference atom:\t"))
x_elec_ref, y_elec_ref, z_elec_ref = supercell_xyz[electrode_atom_ref-1]
x_mol_ref, y_mol_ref, z_mol_ref = x_rxy[molecule_atom_ref-1],y_rxy[molecule_atom_ref-1],z_coord[molecule_atom_ref-1]
################# x,y,z displacement for molecule ################
xt = x_rxy + xmax_sc + abs(x_mol_ref)
yt = y_rxy + y_elec_ref + abs(y_mol_ref)
zt = z_coord + z_elec_ref + abs(z_mol_ref)
###################################################################
if right_electrode == 1:
    Natoms3 = 2*Natoms + Natoms2
elif right_electrode == 2:
    Natoms3 = Natoms + Natoms2
else:
    print("Choose a valid option")
    exit() 

Electrode reference atom:	55
Molecule reference atom:	18


In [10]:
############################################################################################################################################################
################################### For scan distance #####################################################################################################
############################################################################################################################################################
if traslation_rotation == 1:
    print("Scan distance will be do")
    lista = open("lista_scan","w")
    for m in range(len(x_scan)):
        print("Use the next lattice vectors for scattering region: \n")
        if right_electrode == 1:
            x_corr = 0.16901
            print("Left and right electrodes will be draw! \n")
            a1_scatter = [0.5*a0 + np.amax(supercell_xyz[:,0]) + np.amax(xt) + 2*x_scan[m]-x_corr,0.0,0.0]
            a2_scatter = [0.0,3*ymax_sc,0.0]
            a3_scatter = [0.0,0.0,3*zmax_sc]
        if right_electrode == 2:
            print("Only left electrode will be draw! \n")
            a1_scatter = [3*(np.amax(xt) + x_scan[m]), 0.0, 0.0]
            a2_scatter = [0.0, 3*ymax_sc, 0.0]
            a3_scatter = [0.0, 0.0, 3*zmax_sc]
        lattice_vectors = "cell= " + str("{:.5f}".format(a1_scatter[0])) + " 0.00000 0.00000 " + "0.00000 " + str("{:.5f}".format(a2_scatter[1])) + " 0.00000" + " 0.00000 0.00000 " + str("{:.5f}".format(a3_scatter[2])) + " nsc= 1 1 1" + "\n"
        print(lattice_vectors)
        lista = open("lista_scan","a")
        lista.write(str("{:.1f}".format(x_scan[m])) + "\n")
        scatter = open(str(label)+str(Natoms)+"_"+str(molecule_label)+"_d"+str("{:.1f}".format(x_scan[m]))+".xyz","w")
        scatter.write("   " + str(Natoms3) + "\n")
        scatter.write(lattice_vectors)
        for k in range(int(len(supercell_xyz))):
            scatter.write("Au" + "  " + str("{:.5f}".format(supercell_xyz[k,0])) + "  " +str("{:.5f}".format(supercell_xyz[k,1])) + "  " +str("{:.5f}".format(supercell_xyz[k,2])) + "\n")
        for atom in range(Natoms2):
            x = xt[atom] + x_scan[m]
            y = y_rxy[atom] + y_elec_ref - abs(y_mol_ref)
            z = zt[atom]
            scatter.write(str(atom_s[atom]) + "  " + str("{:.5f}".format(x)) + "  " +str("{:.5f}".format(y)) + "  " +str("{:.5f}".format(z)) + "\n")
        if right_electrode == 1:
            for k in range(int(len(supercell_xyz))):
                scatter.write("Au" + "  " + str("{:.5f}".format(supercell_xyz[k,0]+np.amax(xt)+2*x_scan[m]-x_corr)) + "  " +str("{:.5f}".format(supercell_xyz[k,1])) + "  " +str("{:.5f}".format(supercell_xyz[k,2])) + "\n")
        elif right_electrode == 2:
            pass
        scatter.close()
        lista.close()
        SystemXYZ = sisl.io.xyzSile(str(label)+str(Natoms)+"_"+str(molecule_label)+"_d"+str("{:.1f}".format(x_scan[m]))+".xyz",mode='r')
        System    = SystemXYZ.read_geometry()
        System.write(str(label)+str(Natoms)+"_"+str(molecule_label)+"_d"+str("{:.1f}".format(x_scan[m]))+"_geom"+".fdf")
        #System.write("STRUCT_DEVICE_d"+str("{:.1f}".format(x_scan[m]))+".xyz")
        #Device = Readxyz("STRUCT_DEVICE_d"+str("{:.1f}".format(x_scan[m]))+".xyz")
        #Lead   = Readxyz("STRUCT_ELEC.xyz")
        #xD,yD,zD  = Device["x"],Device["y"],Device["z"]
        #xL,yL,zL  = Lead["x"] - a1_elec[0],\
        #        Lead["y"] - a1_elec[1],\
        #        Lead["z"] - a1_elec[2]
        #xR,yR,zR  = Lead["x"] + a1_scatter[0],\
        #        Lead["y"] + a1_scatter[1],\
        #        Lead["z"] + a1_scatter[2]
        #maximo = array( [xD.max(),yD.max(),zD.max(),
        #             xL.max(),yL.max(),zL.max(),
        #             xR.max(),yR.max(),zR.max()] ).max()
        #minimo = array( [xD.min(),yD.min(),zD.min(),
        #             xL.min(),yL.min(),zL.min(),
        #             xR.min(),yR.min(),zR.min()] ).min()
        #ipv.figure()
        #s = ipv.scatter(xL, yL, zL, marker='sphere', size=3)
        #s = ipv.scatter(xD, yD, zD, marker='sphere', size=3,color="blue")
        #s = ipv.scatter(xR, yR, zR, marker='sphere', size=3)
        #ipv.xyzlim(minimo, maximo)
        #ipv.show()
############################################################################################################################################################
################################### For scan rotations #####################################################################################################
############################################################################################################################################################
elif traslation_rotation == 2:
    print("Scan rotation will be done")
    lista_rot = open("lista_rot","w")
    for m in range(len(angle_rotationyz)):
        print("Use the next lattice vectors for scattering region: \n")
        if right_electrode == 1:
            x_corr = 0.16901
            print("Left and right electrodes will be draw! \n")
            a1_scatter = [0.5*a0 + np.amax(supercell_xyz[:,0]) + np.amax(xt) + x_fix1 + x_fix2 - x_corr,0.0,0.0]
            a2_scatter = [0.0,3*ymax_sc,0.0]
            a3_scatter = [0.0,0.0,3*zmax_sc]
        if right_electrode == 2:
            print("Only left electrode will be draw! \n")
            a1_scatter = [3*(np.amax(xt) + x_fix1 + x_fix2), 0.0, 0.0]
            a2_scatter = [0.0, 3*ymax_sc, 0.0]
            a3_scatter = [0.0, 0.0, 3*zmax_sc]
        lattice_vectors = "cell= " + str("{:.5f}".format(a1_scatter[0])) + " 0.00000 0.00000 " + "0.00000 " + str("{:.5f}".format(a2_scatter[1])) + " 0.00000" + " 0.00000 0.00000 " + str("{:.5f}".format(a3_scatter[2])) + " nsc= 1 1 1" + "\n"
        print(lattice_vectors)
        z_ryz,y_ryz = rotation_2D(angle_rotationyz[m],z_coord,y_rxy,z_mol_ref,y_mol_ref)
        lista_rot = open("lista_rot","a")
        lista_rot.write(str("{:.1f}".format(angle_rotationyz[m])) + "\n")
        res2 = open(str(label)+str(Natoms)+"_"+str(molecule_label)+"_r"+str("{:.1f}".format(angle_rotationyz[m]))+".xyz","w")
        res2.write("  " + str(Natoms3) + "\n")
        res2.write(str(lattice_vectors))
        for k in range(int(len(supercell_xyz))):
            res2.write("Au" + "   " + str("{:.5f}".format(supercell_xyz[k,0])) + "   " +str("{:.5f}".format(supercell_xyz[k,1])) + "  " +str("{:.5f}".format(supercell_xyz[k,2])) + "\n")
        for atom in range(Natoms2):
            x = xt[atom] + x_fix1
            y = y_ryz[atom] + y_elec_ref - abs(y_mol_ref)
            z = z_ryz[atom] + z_elec_ref #+ abs(z_mol_ref)
            res2.write(str(atom_s[atom]) + "   " + str("{:.5f}".format(x)) + "   " +str("{:.5f}".format(y)) + "   " +str("{:.5f}".format(z)) + "\n")
        if right_electrode == 1:
            for k in range(int(len(supercell_xyz))):
                res2.write("Au" + "   " + str("{:.5f}".format(supercell_xyz[k,0] + np.amax(xt) + x_fix1 + x_fix2 - x_corr)) + "   " +str("{:.5f}".format(supercell_xyz[k,1])) + "   " +str("{:.5f}".format(supercell_xyz[k,2])) + "\n")
        elif right_electrode == 2:
            pass
        res2.close()
        lista_rot.close()
        SystemXYZ = sisl.io.xyzSile(str(label)+str(Natoms)+"_"+str(molecule_label)+"_r"+str("{:.1f}".format(angle_rotationyz[m]))+".xyz",mode='r')
        System    = SystemXYZ.read_geometry()
        System.write(str(label)+str(Natoms)+"_"+str(molecule_label)+"_r"+str("{:.1f}".format(angle_rotationyz[m]))+"_geom"+".fdf")
else:
    exit()

Scan distance will be do
Use the next lattice vectors for scattering region: 

Left and right electrodes will be draw! 

cell= 28.94270 0.00000 0.00000 0.00000 30.60000 0.00000 0.00000 0.00000 30.60000 nsc= 1 1 1

Use the next lattice vectors for scattering region: 

Left and right electrodes will be draw! 

cell= 29.67604 0.00000 0.00000 0.00000 30.60000 0.00000 0.00000 0.00000 30.60000 nsc= 1 1 1

Use the next lattice vectors for scattering region: 

Left and right electrodes will be draw! 

cell= 30.40937 0.00000 0.00000 0.00000 30.60000 0.00000 0.00000 0.00000 30.60000 nsc= 1 1 1

Use the next lattice vectors for scattering region: 

Left and right electrodes will be draw! 

cell= 31.14270 0.00000 0.00000 0.00000 30.60000 0.00000 0.00000 0.00000 30.60000 nsc= 1 1 1

Use the next lattice vectors for scattering region: 

Left and right electrodes will be draw! 

cell= 31.87604 0.00000 0.00000 0.00000 30.60000 0.00000 0.00000 0.00000 30.60000 nsc= 1 1 1

Use the next lattice vectors f

In [9]:
exit()